<h3>Nomes dos integrantes</h3>
<ul>
    <li>José Paulo Marinho - 02201062</li>
    <li>Kalil Bego - 02201046</li>
    <li>Patrick Lessa Teixeira - 02201058</li>
    <li>Victor Alves Barbosa - 02201039</li>
</ul>

<h3>Tratamento do primeiro data set de Cachorros</h3>

In [100]:
import pandas as pd
dataset_dogs = pd.read_csv('../data/NYC_Dog_Licensing_Dataset.csv')

<b>Removendo colunas que não iremos utilizar</b>

In [101]:
dataset_dogs.drop(['RowNumber', 'CensusTract2010', 'NTA', 'CityCouncilDistrict', 'CommunityDistrict', 'CongressionalDistrict', 'StateSenatorialDistrict', 'LicenseIssuedDate', 'LicenseExpiredDate'], axis=1, inplace=True)
dataset_dogs.drop(dataset_dogs.index[67146:122203], axis=0, inplace=True)
dataset_dogs.head(2)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode
0,SHADOW,M,01/01/2000 12:00:00 AM,Beagle,Brooklyn,11236.0
1,ROCCO,M,10/01/2011 12:00:00 AM,Boxer,Brooklyn,11210.0


In [102]:
from datetime import datetime

# substituindo os valores do campo AnimalBirthMonth para retirar a hora deixa somente a data
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'], format="%d/%m/%Y %H:%M:%S %p")
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'])
dataset_dogs['AnimalBirthMonth'] = dataset_dogs['AnimalBirthMonth'].map(lambda x: datetime.strftime(x,"%d/%m/%Y"), na_action='ignore')
dataset_dogs.head(1)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode
0,SHADOW,M,01/01/2000,Beagle,Brooklyn,11236.0


<h3>Tratamento do segundo dataset_dogs de Gatos</h3>

In [103]:
dataset_cats = pd.read_csv('../data/cats.csv')
dataset_cats.head(1)

#removendo colunas que não iremos utilizar
dataset_cats.drop(['Unnamed: 0', 'id', 'url', 'med_photos', 'size', 'age'], axis=1, inplace=True)

import ast
dataset_cats["photos"] = dataset_cats["photos"].apply(ast.literal_eval)
dataset_cats['photos'] = dataset_cats['photos'].map(lambda foto: foto[0]["large"])

dataset_cats = dataset_cats.rename(columns={'photos': 'link_photo'})
dataset_cats.head(1)

,type,gender,coat,breed,link_photo
0,Cat,Female,NaN,Abyssinian,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...


<b>Salvando dataset_dogss</b>

In [104]:
dataset_dogs.to_csv('../data/dogs_clean.csv')
dataset_cats.to_csv('../data/cats_clean.csv')

<h3>União dos datasets "limpados" acima.</h3>

In [105]:
new_dataset_data_dogs = {
    "AnimalName": dataset_dogs["AnimalName"],
    "AnimalGender": dataset_dogs["AnimalGender"],
    "AnimalBirthMonth": dataset_dogs["AnimalBirthMonth"],
    "BreedName": dataset_dogs["BreedName"],
    "Borough": dataset_dogs["Borough"],
    "ZipCode": dataset_dogs["ZipCode"],
    "Type": "Dog",
    "Image": ""
}

new_dataset_data_cats = {
    "AnimalName": None,
    "AnimalGender": dataset_cats["gender"],
    "AnimalBirthMonth": None,
    "BreedName": dataset_cats["breed"],
    "Borough": None,
    "ZipCode": None,
    "Type": dataset_cats["type"],
    "Image": dataset_cats["link_photo"]
}

new_dataset_dogs = pd.DataFrame(new_dataset_data_dogs)
new_dataset_cats = pd.DataFrame(new_dataset_data_cats)
new_dataset_cats.head(2)

union_dataset = new_dataset_dogs.append(new_dataset_cats)

/tmp/ipykernel_41731/1219320005.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  union_dataset = new_dataset_dogs.append(new_dataset_cats)


In [106]:
import csv

dicionario_ceps = {}

with open('../data/ceps.txt', 'r') as file:
    csvreader = csv.reader(file, delimiter=';')
    for linha in csvreader:
        dicionario_ceps[linha[0]] = linha

In [107]:
import numpy as np
np.random.seed(77)

chaves_dicionario = np.array(list(dicionario_ceps.keys()))

def get_cep():
    key = np.random.choice(chaves_dicionario)
    info_cep = dicionario_ceps[f'{key}']
    cep = info_cep[0]
    return cep

def get_cidade(cep):
    return dicionario_ceps[cep][1]

union_dataset["ZipCode"] = union_dataset["ZipCode"].map(lambda _: get_cep())
union_dataset["Borough"] = union_dataset["ZipCode"].map(lambda cep: get_cidade(cep))

union_dataset.tail(2)
union_dataset.to_csv("../data/dataset_union.csv")



<h3>Tratamento das fotos da base</h3>
<ul>
    <li>Obter a foto da URL</li>
    <li>Converter para base64</li>
</ul>

In [108]:
import requests
import base64

def get_base64_from_photo_url(url):
    if not url:
        return None
    
    response = requests.get(url)
    imagem_bytes = response.content
    imagem_base64 = base64.b64encode(imagem_bytes).decode('utf-8')
    return imagem_base64

In [ ]:
def is_base64(s):
    try:
        base64.b64decode(s)
        return True
    except Exception:
        return False

for index, row in union_dataset.iterrows():
    if row["Image"] and row["Type"] == "Cat" and not is_base64(row["Image"]):
        union_dataset.loc[index, 'Image'] = get_base64_from_photo_url(row["Image"])

union_dataset.to_csv("../data/dataset_union.csv")

<h3>Encontrar fotos dos cachorros em outro dataset</h3>

In [111]:
import os

dir_atual = os.getcwd()

dir_destino = os.path.abspath(os.path.join(dir_atual, os.pardir, 'train'))

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]

fotos = set(arquivos_jpg)

In [112]:
dataset_breed_dogs = pd.read_csv('../data/labels.csv')
dataset_breed_dogs.head(1)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull


In [ ]:
import base64
from PIL import Image

def find_image_from_name_dog(breed_name):
    breeds_founded = dataset_breed_dogs[dataset_breed_dogs['breed'].str.contains(breed_name, case=False)]
    if breeds_founded.empty:
        return None, dataset_breed_dogs
    
    first_breed = breeds_founded.iloc[0]
    df = dataset_breed_dogs.drop(breeds_founded.index[0])
    info_breed = first_breed.to_dict()
    indice_arquivo = arquivos_jpg.index(f"{info_breed['id']}.jpg")
    nome_arquivo = arquivos_jpg[indice_arquivo]
    caminho_completo = os.path.join(dir_destino, nome_arquivo)
    img = Image.open(caminho_completo)
    img_bytes = img.tobytes()
    return base64.b64encode(img_bytes).decode('utf-8'), df

for index, row in union_dataset.iterrows():
    image_base64, df = find_image_from_name_dog(row['BreedName'])
    dataset_breed_dogs = df
    if row["Image"] and row["Type"] == "Dog" and not is_base64(row["Image"]):
        union_dataset.loc[index, 'Image'] = image_base64

union_dataset.to_csv("../data/dataset_union.csv")

<h3>Tratar valores nulos nos dados de gatos utilizando média de outro dataset</h3>

In [114]:
dataset_names_cat = pd.read_csv('../data/2022-licenced-cat-names.csv')
dataset_names_cat.head(2)

,names,quantity
0,LUNA,143.0
1,CHARLIE,116.0


In [115]:
union_dataset.fillna(value=np.nan, inplace=True)
union_dataset.tail(2)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,Type,Image
67143,NaN,Male,NaN,Tuxedo,Diadema/SP,09931150,Cat,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...
67144,NaN,Male,NaN,York Chocolate,Campo Grande/MS,79006280,Cat,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...


In [116]:
union_dataset.head(2)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,Type,Image
0,SHADOW,M,01/01/2000,Beagle,Gravataí/RS,94055030,Dog,
1,ROCCO,M,10/01/2011,Boxer,Várzea Grande/MT,78115646,Dog,


In [122]:
import random

fill_functions = [np.nanmedian, np.mean, lambda x: x[0], lambda x: x[1]]

# escolhe aleatoriamente uma função
fill_func = random.choice(fill_functions)

for i, row in union_dataset.iterrows():
    if pd.isna(row['AnimalBirthMonth']) or pd.isnull(row['AnimalBirthMonth']):
        calculate_func = fill_func
        value = calculate_func(row['AnimalBirthMonth'])
        if isinstance(value, float):
            union_dataset.at[i, 'AnimalBirthMonth'] = np.nan
        else:
            union_dataset.at[i, 'AnimalBirthMonth'] = value[0]


TypeError: 'float' object is not subscriptable